In [765]:
import pandas as pd
import numpy as np
import math
import scipy.stats as st

In [766]:
data = pd.read_csv('titanic_data.csv')
data

Survived  Pclass  Sex   Age  Siblings/Spouses Aboard  \
0           0       3    0  22.0                        1   
1           1       1    1  38.0                        1   
2           1       3    1  26.0                        0   
3           1       1    1  35.0                        1   
4           0       3    0  35.0                        0   
..        ...     ...  ...   ...                      ...   
882         0       2    0  27.0                        0   
883         1       1    1  19.0                        0   
884         0       3    1   7.0                        1   
885         1       1    0  26.0                        0   
886         0       3    0  32.0                        0   

     Parents/Children Aboard     Fare  
0                          0   7.2500  
1                          0  71.2833  
2                          0   7.9250  
3                          0  53.1000  
4                          0   8.0500  
..                       ...      ...  
882                        0  13.0000  
883                        0  30.0000  
884                        2  23.4500  
885                        0  30.0000  
886                        0   7.7500  

[887 rows x 7 columns]

In [767]:
y_label = data['Survived'].to_numpy()
X = data.drop(['Survived'], axis=1).to_numpy()
N, D = X.shape
X = np.append(np.array([[1] for i in range(N)]), X, axis=1)
N, D = X.shape

In [768]:
def dot(theta, X, i):
    N, D = X.shape
    ans = 0
    for j in range(D):
        ans = ans + theta[j]*X[i,j]
    return ans

In [769]:
def l(theta, X):
    ans = 0
    N, D = X.shape
    for i in range(N):
        ans = ans \
        -y_label[i]*math.log(1+math.exp(-dot(theta, X, i)))\
        -(1-y_label[i])*math.log(1+math.exp(dot(theta, X, i)))
    return ans

In [770]:
def nabla_l(theta, X):
    N, D = X.shape
    ans = np.zeros(D)
    for i in range(N):
        ans = ans + \
        (y_label[i] - 1/(1+math.exp(-dot(theta, X, i)))) * X[i,]
    return ans

In [771]:
def hessian(theta, X):
    N, D = X.shape
    ans = np.zeros((D,D))
    for i in range(N):
        ans = ans \
        - np.dot(np.asmatrix(X[i,]).T, np.asmatrix(X[i,]))\
        * math.exp(-dot(theta, X, i)) \
        / ((1+math.exp(-dot(theta, X, i)))**2)
    return ans

In [772]:
def GradientAscent(theta, X, epsilon):
    nabla = nabla_l(theta, X)
    nabla_norm = np.linalg.norm(nabla)
    print("theta = ", theta)
    print("nabla_norm = ", nabla_norm, "\n")
    while nabla_norm > epsilon:
        Hessian = hessian(theta, X)
        alpha = np.dot(np.linalg.inv(Hessian), nabla)
        theta = theta - np.array(alpha)[0]
        nabla = nabla_l(theta, X)
        nabla_norm = np.linalg.norm(nabla)
        print("theta = ", theta)
        print("nabla_norm = ", nabla_norm, "\n")
    return theta

In [773]:
# init parameter
N, D = X.shape
theta = np.array([0 for i in range(D)])
epsilon = 1e-1

In [774]:
# fit
theta = GradientAscent(theta, X, epsilon)
print("l(theta) = ", l(theta, X))

theta =  [0 0 0 0 0 0 0]
nabla_norm =  4043.297274997574 

theta =  [ 1.29729102e+00 -7.20135362e-01  2.03094224e+00 -2.47886747e-02
 -2.01165383e-01 -7.71336514e-02  1.61359036e-03]
nabla_norm =  851.9838354569268 

theta =  [ 2.24795287e+00 -1.07328272e+00  2.59294853e+00 -3.90996306e-02
 -3.45601865e-01 -1.03378863e-01  2.50249418e-03]
nabla_norm =  153.64209589402654 

theta =  [ 2.52222201 -1.17135233  2.74707544 -0.04320928 -0.39757009 -0.10654932
  0.00276689]
nabla_norm =  8.466344116460553 

theta =  [ 2.5398976  -1.17763361  2.75724004 -0.04347261 -0.40180868 -0.10650616
  0.00278559]
nabla_norm =  0.030917228852270735 

l(theta) =  -390.46596480557093


In [775]:
# asymptotic
def fisher(theta, X):
    return -hessian(theta, X)

In [776]:
Mean = theta
Sigma = np.linalg.inv(fisher(theta, X))/N
print("mean = ", Mean)
print("sigma = ", Sigma)

mean =  [ 2.5398976  -1.17763361  2.75724004 -0.04347261 -0.40180868 -0.10650616
  0.00278559]
sigma =  [[ 2.85878325e-04 -7.25558561e-05  9.55952012e-06 -3.32097290e-06
  -1.04587893e-05  2.59694307e-06 -5.96884356e-07]
 [-7.25558561e-05  2.40572406e-05 -7.81153266e-06  5.75781830e-07
   5.18080627e-07 -1.87872676e-06  1.92160429e-07]
 [ 9.55952012e-06 -7.81153266e-06  4.52827834e-05 -2.00717986e-07
  -4.26760245e-06 -5.28294738e-06 -5.21408591e-09]
 [-3.32097290e-06  5.75781830e-07 -2.00717986e-07  6.72365193e-08
   2.41725740e-07  3.96455426e-08  7.11843664e-10]
 [-1.04587893e-05  5.18080627e-07 -4.26760245e-06  2.41725740e-07
   1.38180832e-05 -3.77737769e-06 -3.89853962e-08]
 [ 2.59694307e-06 -1.87872676e-06 -5.28294738e-06  3.96455426e-08
  -3.77737769e-06  1.58545626e-05 -6.77220823e-08]
 [-5.96884356e-07  1.92160429e-07 -5.21408591e-09  7.11843664e-10
  -3.89853962e-08 -6.77220823e-08  6.43654223e-09]]


In [794]:
# Example
x = np.array([1,3,0,22,0,3,7.25])
omega = np.dot(np.asmatrix(theta), np.asmatrix(x).T).tolist()[0][0]
y_ = 1/(1+math.exp(-omega))
print('omega = ', omega)
print('y_ = ', y_)

omega =  -2.2487235177890494
y_ =  0.0954596285185124


In [778]:
# derive tau
alpha = 0.05
std_omega = math.sqrt(np.dot(np.dot(np.asmatrix(x), np.linalg.inv(fisher(theta, X))),np.asmatrix(x).T).tolist()[0][0]/N)
tau = st.norm.interval(1-alpha, loc = omega, scale = std_omega)
print('tau = ', tau[1] - omega)
print(tau)

tau =  0.02451864954578653
(-2.273242167334836, -2.224204868243263)


In [792]:
# Learning Significant Features
df = 1
st.chi2.ppf(1-alpha, df)
Sigma = np.linalg.inv(fisher(theta, X))/N
for j in range(D):
    v = (Sigma[j,j])
    print(j,theta[j]*theta[j]/v)
print('alpha =', alpha, ',', st.chi2.ppf(1-alpha, df))

0 22565.823575436818
1 57646.71571685728
2 167886.60214844203
3 28107.75529789129
4 11683.980296794394
5 715.4761881830041
6 1205.543082667493
alpha = 0.05 , 3.841458820694124


In [795]:
# Example
x = np.array([1,3,1,22,0,3,7.25])
omega = np.dot(np.asmatrix(theta), np.asmatrix(x).T).tolist()[0][0]
y_ = 1/(1+math.exp(-omega))
print('omega = ', omega)
print('y_ = ', y_)

omega =  0.5085165237978917
y_ =  0.6244586486812256
